# 1、载入原始图并进行掩膜的初始化

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图像
img1 = cv2.imread("E:/jupyter_notebook_files/ImageAndVideoProcess/test2/2.jpg")
# 创建一个名为lesson的窗口
cv2.namedWindow("lesson")
# 显示图像
cv2.imshow("lesson", img1)
# hold image
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2、对原图像进行去噪并显示 

#### 中值滤波的思想就是比较一定领域内的像素值的大小，取出其中值作为这个领域的中心像素新的值
- ##### 假设对一定领域内的所有像素从小到大进行排序，如果存在孤立的噪声点，比如椒盐噪声（椒噪声——较小的灰度值，呈现的效果是小黑点；盐噪声——较大的灰度值，呈现的效果是小白点），那么从小到大排序的这个数组中，那些孤立的噪声一定会分布在两边（要么很小，要么很大），这样子取出的中值点可以很好地保留像素信息，而滤除了噪声点的影响。


In [5]:
meanValue_img=cv2.medianBlur(img1,3)
#输出处理好的图像
cv2.imwrite("2_1.jpg", meanValue_img)
cv2.imshow('src',img1)
cv2.imshow('out',meanValue_img)
cv2.waitKey()
cv2.destroyAllWindows() 

NameError: name 'img1' is not defined

# 3、设置鼠标回调函数

In [6]:
import cv2
import numpy as np
 
drawing = False  # 当鼠标按下变为True
ix, iy = -1, -1  # 初始化鼠标位置
 
 
def on_mouse(event, x, y, flags, param):  # 创建回调函数
    global ix, iy, drawing
    if event == cv2.EVENT_LBUTTONDOWN:  # 按下左键
        drawing = True  # 开始绘制
        ix, iy = x, y  # 赋予按下时的鼠标坐标
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:  # 当按下左键拖拽鼠标时
        if drawing is True:
            cv2.circle(img2, (x, y), 3, (255, 255, 255), -1) 
    
img2 = cv2.imread("E:/jupyter_notebook_files/ImageAndVideoProcess/test2/2_1.jpg")
cv2.namedWindow('image')  # 创建空窗口
cv2.setMouseCallback('image', on_mouse)  # 将回调函数与窗口绑定
while(1):
    cv2.imshow('image', img2)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # 按ESC退出
        #cv2.imwrite("2_2.jpg", img)
        break
    #按“x”键对图像进行复原
    elif k == ord('x'):
        #print("hello!")
        #img2 = cv2.imread('E:/jupyter_notebook_files/ImageAndVideoProcess/test2/2_1.jpg')  
        
        #方法二：
        #将画的图案提取为掩膜
        #因为图案是白色的，所以将一个灰色的图片，变成要么是白色要么就是黑色。（大于规定thresh值就是设置的最大值（常为255，也就是白色））
        imggray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)     #通过cvtColor库将其转为灰度
        #将灰度值二值化，得到ROI区域掩模
        ret, mask = cv2.threshold(imggray, 253, 255, cv2.THRESH_BINARY)
        
        #下面是两种不同的修复方式
        img2 = cv2.inpaint(img2,mask,3,cv2.INPAINT_TELEA)
        #img2 = cv2.inpaint(img2,mask,3,cv2.INPAINT_NS)
cv2.destroyAllWindows()

# 思考题1

### 根据自适应滤波的基本原理，设计一种可较好保留纹理细节的自适应中值滤波器对图像（5%椒盐噪声）进行恢复

- ##### 中值滤波器受滤波窗口大小影响较大，用于消除噪声和保护图像细节，两者会存在冲突。如果窗口较小，则能较好地保护图像中的一些细节信息，但对噪声的过滤效果就会打折扣；反之，如果窗口尺寸较大则会有较好的噪声过滤效果，但也会对图像造成一定的模糊效果，从而丢失一部分细节信息。
- #### 自适应中值滤波算法进行图像恢复：根据图像处理的空间相关性原则，采用自适应的方法选择不同的滑动窗口大小；
![](https://img-blog.csdnimg.cn/20200610214750788.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzQyNzc5NDIz,size_16,color_FFFFFF,t_70)

In [8]:
# 自适应中值滤波器

import cv2
import numpy as np

def AdaptProcess(src, i, j, minSize, maxSize):

    filter_size = minSize

    kernelSize = filter_size // 2
    rio = src[i-kernelSize:i+kernelSize+1, j-kernelSize:j+kernelSize+1]
    minPix = np.min(rio)
    maxPix = np.max(rio)
    medPix = np.median(rio)
    zxy = src[i, j]

    if (medPix > minPix) and (medPix < maxPix):
        if (zxy.all() > minPix.all()) and (zxy.all() < maxPix.all()):
            return zxy
        else:
            return medPix
    else:
        filter_size = filter_size + 2
        if filter_size <= maxSize:
            return AdaptProcess(src, i, j, filter_size, maxSize)
        else:
            return medPix


def adapt_meadian_filter(img, minsize, maxsize):

    borderSize = maxsize // 2

    src = cv2.copyMakeBorder(img, borderSize, borderSize, borderSize, borderSize, cv2.BORDER_REFLECT)

    for m in range(borderSize, src.shape[0] - borderSize):
        for n in range(borderSize, src.shape[1] - borderSize):
            src[m, n] = AdaptProcess(src, m, n, minsize, maxsize)

    dst = src[borderSize:borderSize+img.shape[0], borderSize:borderSize+img.shape[1]]
    return dst


# 导入原图像
image = cv2.imread('E:/jupyter_notebook_files/ImageAndVideoProcess/test2/3_5.jpg')

imaged = adapt_meadian_filter(image, 2, 9)
cv2.imshow("original", image)
cv2.imshow("adapt_meadian_filter", imaged)

cv2.waitKey()
cv2.destroyAllWindows()



In [22]:
meanValue_img=cv2.medianBlur(image, 3)
#输出处理好的图像
cv2.imshow('src',image)
cv2.imshow('out',meanValue_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
import sys
sys.version

'3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]'